# Exercise 3.7 Subplots with cartopy
prepared by M.Hauser

We already learned how to create subplots in [Exercise 1.4](./../Part1/ex1_4_subplots.ipynb), and we also used this for map plots. However, we suffer from the same problem as when adding colorbars - the axes are shrunk but the figure is not, leaving large gaps between the rows (or columns)...

Note that most of what we show here for georeferenced plots does *NOT* apply for normal subplots.

In [ ]:
import cartopy.crs as ccrs
import cartopy.util as cutil
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
import xarray as xr

%matplotlib inline

In [ ]:
import mplotutils as mpu

## Setup

Usually we want that the background of the figure is white - but here we want to see how large it really is. Therefore, we color it grey.

In [ ]:
%config InlineBackend.print_figure_kwargs = {'facecolor': '0.9'}

## The Problem - example

Let's do a 2 x 2 map plot:

In [ ]:
f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axes = axes.flatten()

for ax in axes:
    ax.coastlines()
    ax.set_global()

There is too much vertical space between the subplots! Again, the issue is that the aspect ratio of a map plot has to be equal, else it would be distorted. Matplotlib then 'shrinks' the axes, but does not shrink the area of the figure.

## What we can try

There are some functions in pyplot that can potentially solve this problem. (However, they don't).

* `f.tight_layout()`, which tries to optimise the room used for the subplots.
* `subplot_adjust`, i.e. setting the distance between the subplots manually

Before we can illustrate these two, we need to get another obstacle out of the way.


## `bbox_inches='tight'`

When you save a figure you can tell matplotlib to remove all white boundary areas using `plt.savefig('figure.pdf', bbox_inches='tight')`. While this sounds like a good idea, the issue is that the final figure size is quite unpredictable. This can distort your font sizes. Also, if a journal asks for a figure with a certain size you cannot use this option.

> `bbox_inches='tight'` is NOT the same as `f.tight_layout()`

However, figures displayed in the notebooks use this option per default (in contrast to figures you save). To better see what figures we create, we need to turn option off, and again, set the background color to a light grey.

In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None, 'facecolor': '0.9'}

When we now redo the plot from above we see what our pdf would look like:

In [ ]:
f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axes = axes.flatten()

for ax in axes:
    ax.coastlines()
    ax.set_global()


So there is not only un-equal distance between the figures, it also has a very large boundary.

## `tight_layout`

As mentioned `tight_layout` tries to optimise the space between the subplots, but does not really help as well:

In [ ]:
f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axes = axes.flatten()

for ax in axes:
    ax.coastlines()
    ax.set_global()
        
f.tight_layout()

### Exercise

 * try the same with an Orthographic projection

In [ ]:
# code here

### Solution

In [ ]:
f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.Orthographic()))

axes = axes.flatten()

for ax in axes:
    ax.coastlines()
    ax.set_global()
    
f.tight_layout()

## `subplot_adjust`

We already got to know `wspace` and `hspace` in [exercise 1.4](Part1/ex1_4_subplots.ipynb#hspace-and-wspace). Using `subplot_adjust` we can manually fine tune the poitioning of the subplots:

* left, right, bottom, and top are measured in figure coordinates, where (0, 0) is in the lower left corner and (1, 1) in the upper right corner
* wspace the amount of width reserved for blank space between subplots, expressed as a fraction of the average axis width, default value = 0.2
* hspace: the amount of height reserved for white space between subplots, expressed as a fraction of the average axis height, default value = 0.2

For a normal plot (i.e. non map plots), we can use them to adjust the figure to our liking: 

In [ ]:
f, axes = plt.subplots(2, 2)
axes = axes.flatten()

for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    

f.subplots_adjust(left=0.1, right=0.95,
                  bottom=0.1, top=0.9,
                  hspace=0., wspace=0.)


Again, because matplotlib 'shrinks' the axes for map plots, setting hspace and wspace to 0 can leave some vertical distance between subplots:

In [ ]:
f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axes = axes.flatten()

for ax in axes:
    ax.coastlines()
    ax.set_global()

f.subplots_adjust(hspace=0, wspace=0)

# Solution

Also for this problem I developped a solution `mpu.set_map_layout`.

`set_map_layout` respects all parameters of `subplot_adjust` (left, right, bottom, top, hspace, and wspace), and the figure width, but *NOT* its heigh. I.e. it calculates the figure height such that all other parameters are respected.

#### Advantages
 * creating a map plot becomes predictable again (e.g. when you set `hspace=0`, the vertical space is actually 0)

#### Disatvantages
 * it only really works if all subplots have the same aspect ratio
 * needs quite some manual adjustments
 
#### Usage

1. Create figure and axes
* Do all the plotting
* use `f.subplots_adjust`
* use `mpu.set_map_layout(axes, width=17)`
* save the figure




In [ ]:
# 1.
f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axes = axes.flatten()

# 2.

for ax in axes:
    ax.coastlines()
    ax.set_global()

# 3.

f.subplots_adjust(hspace=0, wspace=0)

# 4.
mpu.set_map_layout(axes, width=17) # width is in cm

print('Size (w x h):  ', np.round(f.get_size_inches() * 2.54, 2))

# 5.
# plt.savefig(...)

It worked! `hspace=0, wspace=0` is actually respected!

### Exercise

 * run the code below
   * compare the width/ height of both figures

In [ ]:
f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axes = axes.flatten()

for ax in axes:
    ax.coastlines()
    ax.set_global()
    
f.subplots_adjust(hspace=0.3, wspace=0.15, left=0.05, right=0.95, bottom=0.05, top=0.95)

mpu.set_map_layout(axes, width=17)

print('Size (w x h):  ', np.round(f.get_size_inches() * 2.54, 2))

### Exercise

 * use `mpu.set_map_layout`, and `f.subplots_adjust` to create a 'good' looking figure

In [ ]:
f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.NearsidePerspective()))
axes = axes.flatten()

for i, ax in enumerate(axes):
    ax.coastlines()
    ax.set_global()
    ax.set_title('Map ' + str(i), fontsize=18)

# =============
# adjust size


# ============= 
    
print('Size (w x h):  ', np.round(f.get_size_inches() * 2.54, 2))

### Solution

In [ ]:
f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.NearsidePerspective()))
axes = axes.flatten()

for i, ax in enumerate(axes):
    ax.coastlines()
    ax.set_global()
    ax.set_title('Map ' + str(i), fontsize=18)

# =============
# adjust size
f.subplots_adjust(hspace=0.2, wspace=0.1, left=0.05, right=0.95, bottom=0.05, top=0.9)

mpu.set_map_layout(axes)
# =============


print('Size (w x h):  ', np.round(f.get_size_inches() * 2.54, 2))

## Colorbars

`set_map_layout` can be combinded with the manual creation of colorbars. Note that you will have to manually make room for the colorbars, by adjusting left, hspace, etc...

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
LON, LAT = mpu.infer_interval_breaks(lon, lat)

# =====

f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))
axes = axes.flatten()

for ax in axes:
    ax.coastlines()
    ax.set_global()
    h = ax.pcolormesh(LON, LAT, data, vmin=-1, vmax=1, cmap='RdBu_r', transform=ccrs.PlateCarree())
    
    
f.subplots_adjust(hspace=0.2, wspace=0.35, left=0.025, right=0.875, bottom=0.05, top=0.95)

# ===================================

# color bar1
cbar = mpu.colorbar(h, axes[0], pad=0.015, size=0.05)

# color bar2
cbar = mpu.colorbar(h, axes[2], pad=0.015, size=0.05)

# color bar2
cbar = mpu.colorbar(h, axes[1], axes[3], pad=0.015, size=0.05)

# ===================================

mpu.set_map_layout(axes, width=17)

# ===================================

f.get_size_inches() * 2.54


### Exercise

 * take the plot below and make it look good

In [ ]:
f, axes = plt.subplots(1, 2, subplot_kw=dict(projection=ccrs.NearsidePerspective()))
axes = axes.flatten()

for i, ax in enumerate(axes):
    ax.coastlines()
    ax.set_global()
    ax.set_title('Map ' + str(i), fontsize=18)
    
    h = ax.pcolormesh(LON, LAT, data, vmin=-1, vmax=1, transform=ccrs.PlateCarree())
    
# color bar1
cbar = mpu.colorbar(h, axes[1], size=0.1)

# =============
# adjust size

# f.subplots_adjust(hspace=0.2, wspace=0.1, left=0.05, right=0.95, bottom=0.05, top=0.9)

# mpu.set_map_layout(axes)
# =============

print('Size (w x h):  ', np.round(f.get_size_inches() * 2.54, 2))

### Solution

In [ ]:
f, axes = plt.subplots(1, 2, subplot_kw=dict(projection=ccrs.NearsidePerspective()))
axes = axes.flatten()

for i, ax in enumerate(axes):
    ax.coastlines()
    ax.set_global()
    ax.set_title('Map ' + str(i), fontsize=18)
    
    h = ax.pcolormesh(LON, LAT, data, vmin=-1, vmax=1, transform=ccrs.PlateCarree())
    
# color bar1
cbar = mpu.colorbar(h, axes[1], size=0.1)
    
f.subplots_adjust(hspace=0.2, wspace=0.1, left=0.05, right=0.85, bottom=0.05, top=0.85)

mpu.set_map_layout(axes, width=15.24)

print('Size (w x h):  ', np.round(f.get_size_inches() * 2.54, 2))

# Different projections

To combine different projections in subplots, or to combine map and none map plots, you have to use `gridspec`.

In [ ]:
import matplotlib.gridspec as gridspec

In [ ]:
gs = gridspec.GridSpec(2, 2)

ax1 = plt.subplot(gs[0, 0], projection=ccrs.PlateCarree())

ax2 = plt.subplot(gs[0, 1], projection=ccrs.Robinson())

ax3 = plt.subplot(gs[1, :])

ax1.coastlines()
ax2.coastlines()

x = np.arange(0, 10, 0.1)
ax3.plot(x, np.sin(x))
ax3.plot(x, np.cos(x))

Like this we can create a rotating earth:

In [ ]:
import cartopy.feature as cfeature

In [ ]:
# n_row subplots
n_row = 10

f = plt.figure()
gs = gridspec.GridSpec(1, n_row)

axes = list()

# loop from 0...9
for i in range(n_row):
    
    # rotate
    c_lon = 360. / n_row * i
    
    # create subplot
    ax = plt.subplot(gs[i], projection=ccrs.Orthographic(central_longitude=c_lon, central_latitude=0))
    
    # add some features
    ax.coastlines()
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.LAND)

    # collect the axes
    axes.append(ax)

# adjust the size    
f.subplots_adjust(wspace=0.1, left=0.025, right=0.975)    
mpu.set_map_layout(np.asarray(axes), width=40)

# Bonus: Correct distance between subplots, alternative Solution

As for the colorbars, there is a second solution to overcome the problem, and again it is by using `axes_grid1`:

"[axes_grid1](https://matplotlib.org/2.0.2/mpl_toolkits/axes_grid/users/overview.html) is a collection of helper classes to ease displaying (multiple) images with matplotlib. In matplotlib, the axes location (and size) is specified in the normalized figure coordinates, which may not be ideal for displaying images that needs to have a given aspect ratio."

 > However, it is not part of the core matplotlib functionality, and not it's best-documented part

We will use an [example from the cartopy gallery](http://scitools.org.uk/cartopy/docs/v0.15/examples/axes_grid_basic.html).

In [ ]:
from mpl_toolkits.axes_grid1 import AxesGrid
from cartopy.mpl.geoaxes import GeoAxes

The syntax is not super simple..., `axes_pad=[hspace, wspace]` defines the distance between the subplots in inches.

In [ ]:
projection = ccrs.PlateCarree()
axes_class = (GeoAxes,
              dict(map_projection=projection))


f = plt.figure()
axgr = AxesGrid(f, 111, axes_class=axes_class,
                nrows_ncols=(3, 2),
                axes_pad=[2 / 2.54, 1 / 2.54],
                label_mode='')  # note the empty label_mode

axes = axgr.axes_all

for ax in axes:
    ax.coastlines()    
    ax.set_global()
    
plt.savefig('AxesGrid.pdf')

This may look wrong - but the distance between the subplots are exactly 2 cm & 1 cm (see the AxesGrid.pdf).

While hspace and wspace are correct, this comes at the cost that left, right, top, and bottom may not be set as specified.

### Exercise

 * set axes_pad to 0.5 cm
 * set the figure size to 16 cm x 13 cm (`f.set_size_inches`)
 * play around with `f.subplots_adjust` to find a good setting

In [ ]:
projection = ccrs.PlateCarree()
axes_class = (GeoAxes,
              dict(map_projection=projection))

f = plt.figure()

# set figure size
# f.set_size_inches(...)

axgr = AxesGrid(f, 111, axes_class=axes_class,
                nrows_ncols=(3, 2),
                axes_pad=[2 / 2.54, 1 / 2.54],
                label_mode='')  # note the empty label_mode

axes = axgr.axes_all

for ax in axes:
    ax.coastlines()    
    ax.set_global()
    
# adjust the subplots
# f.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)

### Solution

In [ ]:
projection = ccrs.PlateCarree()
axes_class = (GeoAxes,
              dict(map_projection=projection))


f = plt.figure()

# set figure size
f.set_size_inches(16 / 2.54, 13 / 2.54)

axgr = AxesGrid(f, 111, axes_class=axes_class,
                nrows_ncols=(3, 2),
                axes_pad=[0.5 / 2.54, 0.5 / 2.54],
                label_mode='')  # note the empty label_mode

axes = axgr.axes_all

for ax in axes:
    ax.coastlines()    
    ax.set_global()
    
f.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.95)

## Colorbars

You can also directly add colorbars.

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
LON, LAT = mpu.infer_interval_breaks(lon, lat)

# =====

f = plt.figure()

f.set_size_inches(16 / 2.54, 14.5 / 2.54)


projection = ccrs.PlateCarree()
axes_class = (GeoAxes,
              dict(map_projection=projection))

axgr = AxesGrid(f, 111, axes_class=axes_class,
                nrows_ncols=(3, 2),
                axes_pad=[0.5 / 2.54, 0.5 / 2.54],
                cbar_location='bottom',
                cbar_mode='single',
                cbar_pad=0.1 / 2.54,
                cbar_size='10%',
                label_mode='')  # note the empty label_mode


    
axes = axgr.axes_all
for ax in axes:
    ax.coastlines()
    h = ax.pcolormesh(LON, LAT, data, vmin=-1, vmax=1, cmap='RdBu_r', transform=ccrs.PlateCarree())
    ax.set_global()

axgr.cbar_axes[0].colorbar(h)
    
f.subplots_adjust(left=0.05, right=0.95, bottom=0.1, top=0.95)    

Note how the figure size and bottom had to be adjusted.

### Exercise

 * plot the same data on 3 subplots below each other
 * choose an Orthographic projection
 * add a colorbar for each plot, on the right side of the subplot
 * bonus: adjust the figure size and the subplot params 

In [ ]:
AxesGrid?

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
LON, LAT = mpu.infer_interval_breaks(lon, lat)

# =====

# code here




### Solution

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
LON, LAT = mpu.infer_interval_breaks(lon, lat)

# =====

projection = ccrs.Orthographic()
axes_class = (GeoAxes,
              dict(map_projection=projection))

f = plt.figure()

f.set_size_inches(8 / 2.54, 14.5 / 2.54)

axgr = AxesGrid(f, 111, axes_class=axes_class,
                nrows_ncols=(3, 1),
                axes_pad=0.75 / 2.54,
                cbar_location='right',
                cbar_mode='each',
                cbar_pad=0.5 / 2.54,
                cbar_size='10%',
                label_mode='')  # note the empty label_mode


    
axes = axgr.axes_all
for i, ax in enumerate(axes):
    ax.coastlines()
    h = ax.pcolormesh(LON, LAT, data, vmin=-1, vmax=1, cmap='RdBu_r', transform=ccrs.PlateCarree())
    ax.set_global()
    
    axgr.cbar_axes[i].colorbar(h)
    

f.subplots_adjust(left=0.05, right=0.9, bottom=0.05, top=0.95) 